In [9]:
#pastas colocadas e que funcionam:
#seguenotasfiscais2quinzenadedezembropnz funcionando<<
#n_f52transf_jpajabjpa (deve funcionar mas só 1 file)
#n_fs1718jaymejosedelimajunio (deve funcionar mas só 2 file)
#nfs_henrique_roberval_josrogrio funcionando<<
#nfs_jarbiana não_funcionando
#nfsref2quinzenadezembro2023 tem que ser feito com o leitor de imagem
#ren_fs535455e56edvaldodesouzasilvajunior funcionando <<<

In [37]:
import pdfplumber
import os
from os import *

def leitor(dir):
    fileExt = r".pdf"
    files = os.listdir(dir)

    for arq in files:
        with pdfplumber.open(f"{dir}/{arq}") as pdf:
                text = ""
                for page in pdf.pages:
                    text += page.extract_text()
                    a = text
                    lines = a.split('\n')
                    cda=False
                    tds=False
                    vds=False
                    nfs=False
                    for line in lines:
                        if nfs == True:
                            numero_nota= line.split(' ')[0]
                            nfs=False
                        if vds == True:
                            valor_servico=line.split(' ')[0]
                            vds=False
                        if tds== True:
                            cnpj_tomador = line.split(' ')[0]
                            tds = False
                        if cda == True:
                            chave = line.replace(' ','')[0:44]
                            cda=False
                        if 'ChavedeAcessodaNFS' in line:
                            cda=True
                        
                        if 'PrestadordoServiço' in line:
                            cnpj_prestador = line.split(' ')[1]
                        if 'TOMADORDOSERVIÇO' in line:
                            tds=True
                            
                        if 'ValordoServiço' in line:
                            vds=True
                        if 'NúmerodaNFS' in line:
                            nfs=True
                    print(chave,cnpj_prestador,cnpj_tomador,valor_servico,numero_nota)


In [36]:

cont = 0
with pdfplumber.open(r"C:\Users\jvict\Documents\seven\seven\seguenotasfiscais2quinzenadedezembropnz\BERNEVLDO SEGUND AQUINZENA DE DEZEMBRO.pdf") as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
                a = text
                lines = a.split('\n')
                cda=False
                tds=False
                vds=False
                nfs=False
                for line in lines:
                    if nfs == True:
                        numero_nota= line.split(' ')[0]
                        nfs=False
                    if vds == True:
                        valor_servico=line.split(' ')[0]
                        vds=False
                    if tds== True:
                        cnpj_tomador = line.split(' ')[0]
                        tds = False
                    if cda == True:
                        chave = line.replace(' ','')[0:44]
                        cda=False
                    if 'ChavedeAcessodaNFS' in line:
                        cda=True
                    
                    if 'PrestadordoServiço' in line:
                        cnpj_prestador = line.split(' ')[1]
                    if 'TOMADORDOSERVIÇO' in line:
                        tds=True
                        
                    if 'ValordoServiço' in line:
                        vds=True
                        
                    if 'NúmerodaNFS' in line:
                        nfs=True
                    print(line,cont)
            print(chave,cnpj_prestador,cnpj_tomador,valor_servico,numero_nota)

MUNICIPIODEPETROLINA 0
DANFSev1.0 PREFEITURAMUNICIPALDEPETROLINA 0
DocumentoAuxiliardaNFS-e (87)3861-6676 0
tributos@petrolina.pe.gov.br 0
ChavedeAcessodaNFS-e 0
26111012236240873000119000000000000924018790993190 0
NúmerodaNFS-e CompetênciadaNFS-e DataeHoradaemissãodaNFS-e 0
9 02/01/2024 02/01/202408:34:48 0
NúmerodaDPS SériedaDPS DataeHoradaemissãodaDPS 0
9 900 02/01/202408:34:48 AautenticidadedestaNFS-epodeserverificada 0
pelaleituradestecódigoQRoupelaconsultada 0
chavedeacessonoportalnacionaldaNFS-e 0
EMITENTEDANFS-e CNPJ/CPF/NIF InscriçãoMunicipal Telefone 0
PrestadordoServiço 36.240.873/0001-19 - (87)9811-1647 0
Nome/NomeEmpresarial E-mail 0
BERNEVALDORIBEIROGOMES74792350468 NENEBENE1@HOTMAIL.COM 0
Endereço Município CEP 0
JADE,311 Petrolina-PE 56317-125 0
SimplesNacionalnaDatadeCompetência RegimedeApuraçãoTributáriapeloSN 0
Optante-MicroempreendedorIndividual(MEI) - 0
TOMADORDOSERVIÇO CNPJ/CPF/NIF InscriçãoMunicipal Telefone 0
10.580.938/0005-05 - - 0
Nome/NomeEmpresarial E-mail 

In [38]:
leitor('C:/Users/jvict/Documents/seven/seven/seguenotasfiscais2quinzenadedezembropnz')

26111012236240873000119000000000000924018790 36.240.873/0001-19 10.580.938/0005-05 R$900,00 9
26111012252685890000116000000000000924016396 52.685.890/0001-16 10.580.938/0005-05 R$540,00 9
26111012252685890000116000000000000824013952 52.685.890/0001-16 10.580.938/0005-05 R$2.700,00 8
26111012252685890000116000000000000824013952 52.685.890/0001-16 10.580.938/0005-05 R$2.700,00 8
26111012252685890000116000000000000824013952 52.685.890/0001-16 10.580.938/0005-05 R$2.700,00 8
26111012252685890000116000000000000824013952 52.685.890/0001-16 10.580.938/0005-05 R$2.700,00 8
26111012252685890000116000000000000824013952 52.685.890/0001-16 10.580.938/0005-05 R$2.700,00 8
26051522207246464000121000000000002424010697 07.246.464/0001-21 10.580.938/0005-05 R$1.205,15 24
26051522207246464000121000000000002424010697 07.246.464/0001-21 10.580.938/0005-05 R$1.205,15 24
26051522207246464000121000000000002424010697 07.246.464/0001-21 10.580.938/0005-05 R$1.205,15 24
26051522207246464000121000000000002424010

In [39]:
leitor(r'C:\Users\jvict\Documents\seven\seven\n_f52transf_jpajabjpa')

25075072223159548000184000000000005224014953 23.159.548/0001-84 10.580.938/0010-72 R$2.808,00 52


In [40]:
dir =r'C:\Users\jvict\Documents\seven\seven\nfs_henrique_roberval_josrogrio'
leitor(dir)


26041062248923610000193000000000000324013500 48.923.610/0001-93 10.580.938/0008-58 R$2.800,00 3
26041062248923610000193000000000000424018520 48.923.610/0001-93 10.580.938/0008-58 R$112,68 4
26041062217200875000130000000000000524017021 17.200.875/0001-30 10.580.938/0008-58 R$305,00 5
26041062217200875000130000000000000624019608 17.200.875/0001-30 10.580.938/0008-58 R$2.592,50 6
26041062217200875000130000000000000724013233 17.200.875/0001-30 10.580.938/0008-58 R$200,00 7
26041062232343325000163000000000000424014066 32.343.325/0001-63 10.580.938/0008-58 R$2.592,50 4
26041062232343325000163000000000000524015464 32.343.325/0001-63 10.580.938/0008-58 R$152,50 5
26041062232343325000163000000000000624019450 32.343.325/0001-63 10.580.938/0008-58 R$50,00 6


In [41]:
dir = r"C:\Users\jvict\Documents\seven\seven\nfs_jarbiana"
leitor(dir)

UnboundLocalError: local variable 'chave' referenced before assignment

In [42]:
leitor(r'C:\Users\jvict\Documents\seven\seven\nfsref2quinzenadezembro2023')

UnboundLocalError: local variable 'chave' referenced before assignment

In [43]:
leitor(r"C:\Users\jvict\Documents\seven\seven\ren_fs535455e56edvaldodesouzasilvajunior")

25075072223159548000184000000000005324017802 23.159.548/0001-84 10.580.938/0010-72 R$1.530,00 53
25075072223159548000184000000000005424012535 23.159.548/0001-84 10.580.938/0010-72 R$100,00 54
25075072223159548000184000000000005524010476 23.159.548/0001-84 10.580.938/0010-72 R$2.970,00 55
25075072223159548000184000000000005624011233 23.159.548/0001-84 10.580.938/0010-72 R$330,00 56
